In [1]:
import math

In [2]:
class pathPoint():
    def __init__(self, x, y):
        self.x =x
        self.y =y
        self.w = 0
        self.curv = 0
        self.dist =0
        self.vel =0
        self.finVel =0

In [3]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

def distance2(p1, p2):
    return distance(p1.x, p1.y, p2.x, p2.y)

In [4]:
class vect():
    def __init__(self, x1, y1, x2, y2):
        self.mag = distance(x1, y1, x2, y2)
        if (x2-x1==0):
            if (y2>y1):
                self.angle=90
            else:
                self.angle=-90
        elif (y2-y1==0):
            if (x2>x1):
                self.angle=0
            else:
                self.angle=180
        else:
            self.angle =  math.degrees(math.atan2(y2-y1,x2-x1))

In [5]:
def add(p, vec, t):
        x_temp =p.x + (vec.mag*math.cos(math.radians(vec.angle)))*t
        y_temp =p.y + (vec.mag*math.sin(math.radians(vec.angle)))*t
        return pathPoint(x_temp, y_temp)

In [6]:
points = []
points.append(pathPoint(0, 0))
points.append(pathPoint(50,20))
points.append(pathPoint(100, 100))
points.append(pathPoint(0, 100))

In [7]:
spacing = 1
newPoints = []

for i in range(0,len(points)-1):
    v = vect(points[i].x, points[i].y, points[i+1].x, points[i+1].y)
    num = math.ceil(v.mag/spacing)
    v.mag=spacing
    for j in range(0, num):
        newPoints.append(add(points[i],v,j))
newPoints.append(points[len(points)-1])
        

In [8]:
# for p in newPoints:
#     print(str(p.x)+ "\t" + str(p.y))

In [9]:
def copy(p):
    temp = []
    for t in p:
        temp.append(pathPoint(t.x, t.y))
    return temp

In [10]:
b = 0.75 #weight_smooth: b = 0.75 ~ 0.98
a = 1-b #weight_data: a = 1 - b
tol = 0.001
def smoother(path, a, b, tol):
    path_new = copy(path)
    change = tol
    while (change>=tol):
        change = 0.0
        for i in range(1, len(path)-1):
            aux = path_new[i].x
            path_new[i].x += a* (path[i].x-path_new[i].x) + b*(path_new[i-1].x + path_new[i+1].x - (2.0*path_new[i].x))
            change+= abs(aux-path_new[i].x)
            
            aux = path_new[i].y
            path_new[i].y += a* (path[i].y-path_new[i].y) + b*(path_new[i-1].y + path_new[i+1].y - (2.0*path_new[i].y))
            change+= abs(aux-path_new[i].y)
            
    return path_new

In [11]:
path_Smooth = smoother(newPoints, a, b, tol)

In [12]:
for p in path_Smooth:
    print(str(p.x)+ "\t" + str(p.y))

0.0	0.0
0.0030038415633027106	0.9981421855478382
0.007004099728947676	1.995667572511869
0.01334151485879918	2.9917498107095195
0.024133644844901356	3.985080875546443
0.042977995671148975	4.973436620149966
0.07615246764467512	5.952935795521735
0.13471092521312875	6.916745790262361
0.23817012768215642	7.852804328656747
0.421016014784648	8.739798203233493
0.7441983749387041	9.540058775169456
1.3154462835698744	10.187005957689417
2.027034754937181	10.74721698073286
2.8180141160492562	11.258361760681508
3.6539226934490387	11.741721722993688
4.5151865652884755	12.209457968826058
5.390873232869758	12.6681386223952
6.274620553505414	13.122003026578483
7.162880706945091	13.572908919922272
8.053671070622581	14.022176003530147
8.945751079980072	14.47059686955902
9.838298313586227	14.918525966844275
10.730695660694973	15.366147692587603
11.622292798296545	15.813608013345647
12.512152919411136	16.26102059013932
13.398739232928932	16.7084600171624
14.279435495160273	17.15599622995765
15.149639313873

In [13]:
path_Smooth[0].dist =0
for i in range(1, len(path_Smooth)):
    path_Smooth[i].dist = path_Smooth[i-1].dist + distance(path_Smooth[i].x, path_Smooth[i].y, path_Smooth[i-1].x, path_Smooth[i-1].y)

In [14]:
# for p in path_Smooth:
#     print(str(p.dist))

In [15]:
def area_of_triangle(a, b, c):
    return abs((b.x-a.x)*(c.y-a.y) - (b.y-a.y)*(c.x-a.x));

In [16]:
path_Smooth[0].curv=0.0001 #add to not /0 error
path_Smooth[-1].curv=0.0001
for i in range(1, len(path_Smooth)-1): #Menger curvature
    path_Smooth[i].x+=0.0001
    path_Smooth[i].y+=0.0001
    p1 = path_Smooth[i-1]
    p2 = path_Smooth[i]
    p3 = path_Smooth[i+1]
    temp = (4*area_of_triangle(p1, p2, p3))
    path_Smooth[i].curv = temp/(distance2(p1, p2)* distance2(p2, p3)* distance2(p3, p1))
    if (path_Smooth[i].curv==0):
        path_Smooth[i].curv+=0.001

In [17]:
# for p in path_Smooth:
#     print(str(p.curv))

In [18]:
max_vel = 200 #speed
turning_const = 5 #changes how fast it goes around turns
for p in path_Smooth:
    p.vel = min(max_vel, turning_const/p.curv)

In [19]:
#max accel
max_accel = 70
starting_vel = 36
path_Smooth[-1].vel =0
for i in range(len(path_Smooth)-2, -1, -1):
    d = distance2(path_Smooth[i+1], path_Smooth[i])
    path_Smooth[i].finVel = min(path_Smooth[i].vel, math.sqrt(path_Smooth[i+1].finVel**2 + 2*max_accel*d))


path_Smooth[0].finVel = starting_vel
for i in range(1, len(path_Smooth)):
    d = distance2(path_Smooth[i-1], path_Smooth[i])
    test = math.sqrt(path_Smooth[i-1].finVel**2 + 2*max_accel*d)
    if (test<path_Smooth[i].finVel):
        path_Smooth[i].finVel=test
    else:
        break

TO DO: RATE LIMITER

In [24]:
# for p in path_Smooth:
#     print(str(p.curv))

0	36

1	35.1864524874173

2	33.1425973869799

3	30.967360652475563

4	28.633946100815603

5	26.105706594861314

6	23.33022512136542

7	20.22664541517901

8	16.65145551747843

9	12.267062983962848

10	6.385266962913162

11	12.261071337413693

12	24.77152285757305

13	48.186731712875634

14	49.08261545968739

15	47.664370729503595

16	46.18978644123506

17	44.65880866814907

18	43.0691034442652

19	41.41586246986055

20	39.69246367676623

21	37.89039626939342

22	35.9987588609182

23	34.003667681691084

24	31.887434809182636

25	29.627271611783875

26	27.193224880699077

27	24.545150663635646

28	21.627152796436416

29	18.357753133594098

30	14.603228579389715

31	10.072200564861943

32	3.0133691869106

33	2.0148287555138586

34	7.334511141997353

35	21.8360359089213

36	51.1589466836717

37	58.436150550035336

38	57.280907273694396

39	56.07752666559323

40	54.833583780581236

41	53.55252892598572

42	52.23528933027435

43	50.88118554399107

44	49.48843481150952

45	48.0544054559955

46

In [44]:
f = open("data.csv", "w")
for p in path_Smooth:
    f.write(str(p.x) + "," + str(p.y) + "," + str(p.finVel) + "\n")
    print(str(p.x) + "\t" + str(p.y))

0.0	0.0
0.9285766908852593	0.3714906763541037
1.8570533378922076	0.7428813351568834
2.7855304576319506	1.114272385653948
3.714005576864447	1.4856600696666595
4.642484244805225	1.8570632680133436
5.570963933060549	2.228423808137235
6.499427759688273	2.5998548146115725
7.427918205020484	2.971224308280447
8.35639603804955	3.3426024446753315
9.28486292907254	3.7140085296490883
10.213338365536453	4.085407365679686
11.141821452136732	4.4567899967905
12.07030149802763	4.828172555819662
12.998775531485713	5.199564583961496
13.927248295150921	5.570961388677369
14.85572407524156	5.942355259790837
15.784202771581187	6.3137442066467075
16.712681760459258	6.685131334655083
17.64115910031374	7.056520116782048
18.569634802000838	7.42791147799272
19.498110072037363	7.799304108080834
20.42658600587146	8.170696268017654
21.35506289096048	8.54208713605805
22.28354031853498	8.913476994578081
23.21201768572275	9.28486664582459
24.14049461658006	9.656256747998599
25.06897108869668	10.027647497403741
25.9974